In [17]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import EvolvedOperatorAnsatz
from qiskit.quantum_info import SparsePauliOp
# from qiskit.quantum_info import Pauli, Statevector
from qiskit_nature.second_q.operators import FermionicOp
from qiskit.circuit.library import UnitaryGate
# from qiskit.primitives import Estimator

In [18]:
n = 4
N = 2*n
t = 1
mu = 1.5
U = 3

In [19]:
def qOp(i,j):
    return FermionicOp(
    {
        "+_{i} -_{j}".format(i=i%N,j=j%N): 1.0,
    },
    num_spin_orbitals=N,
)

In [20]:
t_list = []
U_list = []
for i in range(n):
    t_list.append((2*i,(2*i+2)%N))
    t_list.append((2*i+1,(2*i+3)%N))
site_list = [2*i for i in range(n)]

t_term = 0
U_term = 0
mu_term = 0

for edge in t_list:
    t_term += qOp(edge[0],edge[1])
    t_term += qOp(edge[1],edge[0])

for u in site_list:
    mu_term += qOp(u,u) + qOp(u+1,u+1)
    U_term += qOp(u,u)@qOp(u+1,u+1)

H = U * U_term - mu * mu_term - t * t_term

U_list = [(i,i+1) for i in site_list]

In [21]:
def makePauliStr(N,idx,gs):
    pauli_str = ""
    for i in range(N):
        if i in idx:
            pauli_str+=gs[idx.index(i)]
        else:
            pauli_str+="I"
    return pauli_str

def ZZ(N,i,j):
    zz_str = makePauliStr(N,(i,j),["Z","Z"])
    return SparsePauliOp([zz_str],coeffs = [.5])

def dZZ(N,i,j,k,l):
    zz_str = makePauliStr(N,(i,j,k,l),["Z","Z","Z","Z"])
    return SparsePauliOp([zz_str],coeffs = [.5])

def XXplusYY(N,i,j):
    xx_str = makePauliStr(N,(i,j),["X","X"])
    yy_str = makePauliStr(N,(i,j),["Y","Y"])
    return SparsePauliOp([xx_str,yy_str],coeffs = [.25,.25])

def dXXplusYY(N,i,j,k,l):
    ij = XXplusYY(N,i,j)
    kl = XXplusYY(N,k,l)
    return ij@kl

In [22]:
def poolXXYY(N,ts):
    pool = []
    for i,edge in enumerate(ts):
        p = XXplusYY(N,edge[0],edge[1])
        # print(isinstance(p,Operator))
        pool.append(p)
    return pool

def pooldXXYY(N,ts):
    pool = []
    for i,edge in enumerate(ts):
        for j in range(i):
            p = dXXplusYY(N,edge[0],edge[1],ts[j][0],ts[j][1])
            # print(isinstance(p,Operator))
            pool.append(p)
    return pool

def poolZZ(N,Us):
    pool = []
    for edge in Us:
        p = ZZ(N,edge[0],edge[1])
        pool.append(p)
    return pool

def pooldZZ(N,Us):
    pool = []
    for i,edge in enumerate(Us):
        for j in range(i):
            p = dZZ(N,edge[0],edge[1],Us[j][0],Us[j][1])
            # print(isinstance(p,Operator))
            pool.append(p)
    return pool

def totalPool(N,Us,ts):
    ZZ = poolZZ(N,Us)
    dZZ = pooldZZ(N,Us)
    XXYY = poolXXYY(N,ts)
    dXXYY = pooldXXYY(N,ts)
    return XXYY + ZZ + dXXYY + dZZ

pool = totalPool(N,t_list,U_list)

In [23]:
li = [i for i in range(0, N // 2, 2)] + [N - 1 - i for i in range(0, N // 2, 2)]
def s_circ(N):
    qc = QuantumCircuit(N)
    print(li)
    qc.x(li)
    qc.barrier()
    return qc

def get_evolved_operator_ansatz(pool, N):
    ans = EvolvedOperatorAnsatz(pool, initial_state=s_circ(N), parameter_prefix="b")
    ans1 = EvolvedOperatorAnsatz(pool, initial_state=ans)
    print_ans = ans.initial_state.decompose()
    print(print_ans)
    return ans, ans1

print(get_evolved_operator_ansatz(pool=pool, N=N))

[0, 2, 7, 5]
     ┌───────────┐ ░ 
q_0: ┤ U3(π,0,π) ├─░─
     └───────────┘ ░ 
q_1: ──────────────░─
     ┌───────────┐ ░ 
q_2: ┤ U3(π,0,π) ├─░─
     └───────────┘ ░ 
q_3: ──────────────░─
                   ░ 
q_4: ──────────────░─
     ┌───────────┐ ░ 
q_5: ┤ U3(π,0,π) ├─░─
     └───────────┘ ░ 
q_6: ──────────────░─
     ┌───────────┐ ░ 
q_7: ┤ U3(π,0,π) ├─░─
     └───────────┘ ░ 
(<qiskit.circuit.library.n_local.evolved_operator_ansatz.EvolvedOperatorAnsatz object at 0x32b6a9910>, <qiskit.circuit.library.n_local.evolved_operator_ansatz.EvolvedOperatorAnsatz object at 0x32b7b2a50>)


In [24]:
i = 1j
theta = 0.7809

def transform_circuit(N):
    qc = QuantumCircuit(N)
    print(li)
    qc.x(li)
    qc.barrier()
    return qc

ans = EvolvedOperatorAnsatz(pool, initial_state=transform_circuit(N), parameter_prefix="b")
ans1 = EvolvedOperatorAnsatz(pool, initial_state=ans)
print(ans.initial_state.decompose())
print(transform_circuit(N))

[0, 2, 7, 5]
     ┌───────────┐ ░ 
q_0: ┤ U3(π,0,π) ├─░─
     └───────────┘ ░ 
q_1: ──────────────░─
     ┌───────────┐ ░ 
q_2: ┤ U3(π,0,π) ├─░─
     └───────────┘ ░ 
q_3: ──────────────░─
                   ░ 
q_4: ──────────────░─
     ┌───────────┐ ░ 
q_5: ┤ U3(π,0,π) ├─░─
     └───────────┘ ░ 
q_6: ──────────────░─
     ┌───────────┐ ░ 
q_7: ┤ U3(π,0,π) ├─░─
     └───────────┘ ░ 
[0, 2, 7, 5]
     ┌───┐ ░ 
q_0: ┤ X ├─░─
     └───┘ ░ 
q_1: ──────░─
     ┌───┐ ░ 
q_2: ┤ X ├─░─
     └───┘ ░ 
q_3: ──────░─
           ░ 
q_4: ──────░─
     ┌───┐ ░ 
q_5: ┤ X ├─░─
     └───┘ ░ 
q_6: ──────░─
     ┌───┐ ░ 
q_7: ┤ X ├─░─
     └───┘ ░ 
